In [ ]:
import pandas as pd
import numpy as np
from itertools import compress

In [ ]:
#neat-ish
fuel=pd.read_excel("data/alt_fueling_stations.xlsx", header=2, usecols=[1,10],skipfooter=20)
elec=pd.read_excel("data/electricity.xlsx", header=1)
hev=pd.read_excel("data/hev_sale.xlsx", header=2, usecols=[1,68],skipfooter=19)

#Sheets need to be combined
meat=pd.ExcelFile("data/meat_consumption.xlsx")
red_meat=pd.read_excel(meat,sheet_name="WASDE_TotalRedMeat-Full",header=2, usecols=[0,1,13],names=["red_meat_per_capita"],skipfooter=5)
white_meat=pd.read_excel(meat,sheet_name="WASDE_TotalPoultry-Full",header=2, usecols=[0,1,13],names=["poultry_per_capita"],skipfooter=5)

#Melt
pev=pd.read_excel("data/pev_sales.xlsx", header=2,skipfooter=10)
######FIND TOTAL VEHICLE SALES DATA SET######

#Melt
waste=pd.read_excel("data/waste_recycling.xlsx",skipfooter=3,index_col=0)

#Sweet Baby Lettuce why (condense and combine)
public_trans = pd.ExcelFile("data/public_trans.xlsx")
cols = [i for i in range(219) if i not in [0,1,2,3,4,5,6,8]]
UPT=pd.read_excel(public_trans,sheet_name="UPT",usecols=cols)
VRM=pd.read_excel(public_trans,sheet_name="VRM",usecols=cols)
VRH=pd.read_excel(public_trans,sheet_name="VRH",usecols=cols)

In [ ]:
years=[x[0] for x in red_meat.index]
i=~np.isnan(years)
years=list(compress(years,i))

red_meat_per_capita=[]
poultry_per_capita=[]
for (i,(year,quart)) in enumerate(red_meat.index):
    if quart=='Yr Jan-Dec':
        red_meat_per_capita=red_meat_per_capita+[red_meat['red_meat_per_capita'].iloc[i]]
        poultry_per_capita=poultry_per_capita+[white_meat['poultry_per_capita'].iloc[i]]
    

meat=pd.DataFrame({'Year':years,'Red':red_meat_per_capita,'White':poultry_per_capita})

In [ ]:
hev.columns=['Year','Hybrids']
PEVs=pev.transpose().iloc[2:9,-1].reset_index()
PEVs.columns=['Year','Plug-ins']

vehicles=hev.merge(PEVs, on='Year', how='outer')

In [ ]:
waste=waste.transpose()[['Products - Total Materials','Total MSW Generated - Weight']].reset_index()
waste.columns=['Year','Total_Recycled','Total_MSW']
waste['Percent_Recycled']=waste['Total_Recycled']/waste['Total_MSW']

In [ ]:
electric=(elec[['Year','Residential', 'Commercial', 'Industrial', 'Transportation','Total']]
          .groupby('Year')
          .sum()
          .reset_index())

In [ ]:
# UPT=pd.read_excel(public_trans,sheet_name="UPT",usecols=cols)

In [ ]:
# UPT=(UPT.transpose().reset_index())

# col_names=UPT.iloc[0]
# UPT.columns=col_names
# UPT = UPT.drop(0)

# month=[]
# year=[]

# for word in UPT['Modes']:
#     month=month+[word[:3]]
#     year=year+[word[3:]]

# UPT['year']=year
# col_names = UPT.columns.tolist()
# col_names = col_names[-1:] + col_names[:-1]
# UPT = UPT[col_names]
# UPT = UPT.drop(columns='Modes')

In [ ]:
# upt_columns = list(UPT.columns.unique())
# upt_columns = [col for col in upt_columns if str(col) != 'nan']
# upt_by_mode = UPT[upt_columns]
# upt_by_mode = upt_by_mode.set_index('year')

In [ ]:
dates = list(UPT.columns)
del dates[0]
melted = UPT.melt(id_vars=['Modes'], value_vars=dates, var_name='Date', value_name='UPT')

year=[]

for word in melted['Date']:
#     if len(year)==0:
#         print('First Element')
#     elif year[len(year)-1]!=word[3:]:
#         print(word[3:])
#     else:
#         pass
    year=year+['20'+word[3:]]

melted['Year']=year
col_names = melted.columns.tolist()
col_names = col_names[-1:] + col_names[:-1]
melted = melted[col_names]
melted = melted.drop(columns='Date')